### Import dependencies

In [5]:
import database
from data_api import youtube
from decouple import config
from pprint import pprint

### Import Google Data API Key and Initiate Youtube API Class Instance
1. Create Youtube Data API key as per the instructions at: https://developers.google.com/youtube/v3/getting-started \

2. Create `.env` file containing the key as follows: `YT_API_KEY=[key]`

In [6]:
db = database.yt_sentiment()
db.update_categories()

categories table updated successfully!


In [7]:
DEVELOPER_KEY = config('YT_API_KEY')
yt = youtube(DEVELOPER_KEY)

In [9]:
yt.category_search(28, search_term="macbook", order="viewCount")

[{'kind': 'youtube#searchListResponse',
  'etag': '_enIEZeRN_OdHcz7734LipJE46A',
  'nextPageToken': 'CDIQAA',
  'regionCode': 'GB',
  'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 50},
  'items': [{'kind': 'youtube#searchResult',
    'etag': 'znDuvct4Fouwvt2oLdS6x0RgxIs',
    'id': {'kind': 'youtube#video', 'videoId': '-s0906zh3bk'},
    'snippet': {'publishedAt': '2015-07-23T09:14:29Z',
     'channelId': 'UCiEDKBFjGOT-zdxU9RApxDA',
     'title': 'Summer CARnage? Dyson V6 has you covered',
     'description': 'Family trips to the beach; fun, picnics, ice cream, swimming. So far so good. But trips to the beach can also leave your car ...',
     'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/-s0906zh3bk/default.jpg',
       'width': 120,
       'height': 90},
      'medium': {'url': 'https://i.ytimg.com/vi/-s0906zh3bk/mqdefault.jpg',
       'width': 320,
       'height': 180},
      'high': {'url': 'https://i.ytimg.com/vi/-s0906zh3bk/hqdefault.jpg',
       'width': 4